In [9]:
import socket
import hashlib
import matplotlib.pyplot as plt
import time
import numpy as np
t0 = time.perf_counter()
serverAddressPort   = ("127.0.0.1", 9801)

bufferSize          = 1500


def sleep(duration, get_now=time.perf_counter):
    now = get_now()
    end = now + duration
    while now < end:
        now = get_now()

In [10]:
UDPClientSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
UDPClientSocket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

In [11]:
print(serverAddressPort)
UDPClientSocket.sendto(str.encode("SendSize\nReset\n\n"), serverAddressPort)
print("sent request")
totalsize = 0

('127.0.0.1', 9801)
sent request


In [12]:
while True:
    try:
        msgFromServer = UDPClientSocket.recvfrom(bufferSize)
        print(msgFromServer)
        msg = msgFromServer[0].decode()
        print(msg)
        brokenmessage = msg.split("\n")
        if(brokenmessage[-1] == "" and brokenmessage[-2] == ""):
            if (brokenmessage[0][:4] == "Size"):
                totalsize = int(brokenmessage[0][6:])
        break
    except TimeoutError:
        time.sleep(0.2)
        UDPClientSocket.sendto(str.encode("SendSize\nReset\n\n"), serverAddressPort)

(b'Size: 888034\n\n', ('127.0.0.1', 9801))
Size: 888034


In [30]:
UDPClientSocket.sendto(str.encode("Offset: " + "0" + "\nNumBytes: "
                                      + str(min(1448, totalsize - 0)) + "\n\n"), serverAddressPort)
msgFromServer = UDPClientSocket.recvfrom(bufferSize)
print(msgFromServer)

(b'Offset: 0\nNumBytes: 1448\n\nby Sir Arthur Conan Doyle\nCopyright laws are changing all over the world. Be sure to check the\nthis or any other Project Gutenberg eBook.\nThis header should be the first thing seen when viewing this Project\nheader without written permission.\neBook and Project Gutenberg at the bottom of this file.  Included is\nimportant information about your specific rights and restrictions in\ndonation to Project Gutenberg, and how to get involved.\n**Welcome To The World of Free Plain Vanilla Electronic Texts**\nAuthor: Sir Arthur Conan Doyle\n[Most recently updated: November 29, 2002]\nLanguage: English\nCharacter set encoding: ASCII\n(Additional editing by Jose Menendez)\nTHE ADVENTURES OF\nCONTENTS\nI.\tA Scandal in Bohemia\nII.\tThe Red-Headed League\nIII.\tA Case of Identity\nIV.\tThe Boscombe Valley Mystery\nV.\tThe Five Orange Pips\nVIII.\tThe Adventure of the Speckled Band\nX.\tThe Adventure of the Noble Bachelor\nI had seen little of Holmes lately. My ma

In [ ]:
counter = 0

def fetch_data(burst_size:int, wait_time:float):
    nonlocal counter
    for i in range(burst_size):
        UDPClientSocket.sendto(str.encode("Offset: " + str(counter) + "\nNumBytes: "
                                      + str(min(1448, totalsize - counter)) + "\n\n"), serverAddressPort)
        while True:
            try:
                msgFromServer = UDPClientSocket.recvfrom(bufferSize)
                msg = msgFromServer[0].decode()
                brokenmessage = msg.split("\n")
                print(counter)
                if(brokenmessage[2] == ""):
                    if (brokenmessage[0][:6] == "Offset"):
                        offset = int(brokenmessage[0][8:])
                        print("offset:" + str(offset))
                        packsize = int(brokenmessage[1][10:])
                        print("packetsize: "+str(packsize))
                        print(len(msg) - (len(brokenmessage[0])+len(brokenmessage[1])+3))
                        if (offset == counter and (len(msg) - (len(brokenmessage[0])+len(brokenmessage[1])+3))== packsize):
                            data += msg[len(brokenmessage[0]) +len(brokenmessage[1])+3:]
                            graph_offset_recv.append(offset)
                            graph_time_recv.append(time.perf_counter() - t0)
                            counter += packsize
                            sleep(0.00245)
                            break
                        else:
                            sleep(0.002)
                            UDPClientSocket.sendto(str.encode("Offset: "+str(counter)+"\nNumBytes: "+str(min(1448,totalsize - counter))+"\n\n"), serverAddressPort)
    
                    else:
                        sleep(0.002)
                        UDPClientSocket.sendto(str.encode("Offset: "+str(counter)+"\nNumBytes: "+str(min(1448,totalsize - counter))+"\n\n"), serverAddressPort)
                else:
                    sleep(0.002)
                    UDPClientSocket.sendto(str.encode("Offset: "+str(counter)+"\nNumBytes: "+str(min(1448,totalsize - counter))+"\n\n"), serverAddressPort)
                
                
            except TimeoutError:
                sleep(0.003)
                UDPClientSocket.sendto(str.encode("Offset: "+str(counter)+"\nNumBytes: "+str(min(1448,totalsize - counter))+"\n\n"), serverAddressPort)
        return (counter < totalsize)